# Формирование модели монетизации

Загрузка, первичный осмотр и предобработка данных:

1. чтение
2. поиск дубликатов
3. поиск пустых значений
4. проверка типов данных
Проведение исследовательского анализа данных:

1. смотрим на данные по стоимости кликам по источникам
2. анализируем влияние событий на совершение целевого события
3. количество пользователей по событиям
4. количество пользователей по заданиям
Сегментация покупателей на основе истории их покупок:

1. деление основной базы на 2 базы с разными событиями
2. вычисление  времени прохождения уровня пользователями с разными событиями
Проверка гипотезы:

1. проверка гипотезы различия времени прохождения уровня между пользователями, которые заканчивают уровень через реализацию проекта, и пользователями, которые заканчивают уровень победой над другим игроком
Презентация:

1. выделение ключевых моментов и краткое описание сути

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import scipy.stats as stats
import seaborn as sns
from scipy import stats as st

In [2]:
df=pd.read_csv('game_actions.csv')

FileNotFoundError: [Errno 2] File b'game_actions.csv' does not exist: b'game_actions.csv'

In [ ]:
df1=pd.read_csv('ad_costs.csv')

In [ ]:
df2=pd.read_csv('user_source.csv')

# Предобработка данных

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df1.describe()

In [ ]:
df1.isna().sum()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.duplicated().sum()

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
df2.describe()

In [ ]:
df2.isna().sum()

In [ ]:
df2.isnull().sum()

In [ ]:
df2.duplicated().sum()

## Разбираемся с типами

In [ ]:
df1['day'] = pd.to_datetime(df1['day'])

In [ ]:
df['event_datetime'] = pd.to_datetime(df['event_datetime'])

## Предобработка готова.

# Вывод: Дубли очищены, пустых значений нет, даты приведены в правильный тип

In [ ]:
df.head()

In [ ]:
df1.head()

In [ ]:
df2.head()

# Вывод: первой базы в полне достаточно для основной работы.

# Исследовательский анализ

### Стоимость кликов по источникам и графики по дням

#### Суммируем доходы по источникам

In [ ]:
df1.groupby('source')['cost'].sum()

### График для наглядности(доходы по источникам) по дням

In [ ]:
dau = df1.groupby(['day', 'source']).agg({'cost': 'sum'}).unstack().plot(figsize=(18,5))
plt.xlabel('Дата')
plt.ylabel('Количество пользователей по дням')
plt.legend(('facebook_ads','instagram_new_adverts','yandex_direct','youtube_channel_reklama'), title='источники')

### График для наглядности(доходы по источникам) по дням #2

In [ ]:
dau

In [ ]:
dau = df1.groupby(['day', 'source']).agg({'cost': 'sum'}).unstack().plot(kind="barh", figsize = (14,9),  fontsize=15)
plt.title('Стоимость кликов по источникам',fontsize=20)
plt.xlabel('Деньги', fontsize=15) 
plt.ylabel('Дата', fontsize=15) 
plt.show()

## Вывод: меньше всего денег ушло на то, чтобы привлечь пользователей с ютуба

 ### Количество уникальных пользователей по источникам

In [ ]:
uni = df2.groupby(['source']).agg({'user_id': 'nunique'})

In [ ]:
uni.reset_index()

In [ ]:
(uni['user_id'].plot(kind='pie', labels=None, label='', autopct='%1.0f%%', 
       legend=True, figsize=(10, 10), title='Разбивка клиентов по источникам', fontsize=20))
plt.show()

## яндекс привёл больше всех пользователей.

In [ ]:
df1.groupby('source')['cost'].sum()

In [ ]:
df2.groupby('source')['user_id'].count()

In [ ]:
face=2140/2726
insta=2161/3347
ya=2233/4817
yout=1068/2686

In [ ]:
face

In [ ]:
insta

In [ ]:
ya

In [ ]:
yout

# Самый эффективный ресурс по привлечению пользователей  это ютуб

In [ ]:
df['event'].unique()

In [ ]:
df['building_type'].unique()

## Далее смотрим чем занимаются пользователи

In [ ]:
aa=df.groupby(['event']).agg({'user_id': 'nunique'}).plot(kind='bar', figsize=(10, 5))
sobitiye = ['building', 'finished_stage_1', 'project']
aa.set_xticklabels(sobitiye, rotation=15)
plt.title('Количество пользователей по событиям',fontsize=30)
aa.set_xlabel('События', fontsize=15)
aa.set_ylabel('Количество',fontsize=15 )
plt.show()

In [ ]:
aa2=df.groupby(['building_type']).agg({'user_id': 'nunique'}).sort_values('user_id').plot(kind='bar', figsize=(10, 5))
sobitiye2 = ['assembly_shop', 'spaceport', 'research_center']
aa2.set_xticklabels(sobitiye2, rotation=15)
plt.title('Количество пользователей по заданиям',fontsize=30)
aa2.set_xlabel('Задания', fontsize=15)
aa2.set_ylabel('Количество',fontsize=15 )
plt.show()

# Выводы: 
## - не много тех, кто заканчивает первый уровень. строят строят и уходят. больше половины таких. 
## - строят в основном космопорты и исследовательские центры

In [ ]:
firts_date = df['event_datetime'].min()
last_date = df['event_datetime'].max()

In [ ]:
firts_date

In [ ]:
last_date

## Ниже я создаю две базы building и project. В одной все люди кто что-то строил и потом закончил уровень(победа над другим игроком). В другой все кто так же что-то строил, окончил проект, и  окончили уровень. Потом считаю сколько времени прошло между ПЕРВЫМ(первая стройка и проект) событием и окончанием уровня.

In [ ]:
SS=df.groupby(['user_id'])['event'].nunique()

In [ ]:
SS=SS.reset_index()

In [ ]:
SS.columns=['user_id','gruppa']

In [ ]:
SS.head(15)

### Вот они все. Объединяем с главное базой.

In [ ]:
grupp=pd.merge(df, SS, on='user_id', how='outer')

In [ ]:
grupp.head()

###  билдинг это группа людей, которые  прошли уровень победой над врагом. проект это группа тех, кто закончил проект

### описываю логику дальнейших действий:
1. ищу первую постройку, группируя по схеме пользователь - событие - минимальная дата(первая постройка)
2. реиндексирую и все дела база готова
3. есть такая штуковина как дифф, которая вычисляет разницу между временами(позднее минус ранее). в нашем случаем это окончание уровня минус первая постройка.  меньшей датой всегда будет первой постройка. 2 события 2 даты. 
4. далее получаем что-то типа этого 6 days 10:38:42. переводим всё в часы.
5. удаляем все билдинги с нанами. расходный материал.
6. часы делаем целочисленными

In [ ]:
building=grupp.query('gruppa==2')

In [ ]:
building = building.groupby(['user_id','event'])['event_datetime'].min()

In [ ]:
building.name = 'first_event_datetime'

In [ ]:
building=building.reset_index()

In [ ]:
building['diff'] = building.groupby('user_id')['first_event_datetime'].diff(1)

In [ ]:
building['diff'] = building['diff']/np.timedelta64(1, 'h')

In [ ]:
building=building.dropna()

In [ ]:
building['diff'] = building['diff'].astype('int')

In [ ]:
building = building.sort_values('user_id')

In [ ]:
building.head()

##  так как у проекта 3 события то чтобы дифф сработал просто выбрасываем строки с project. 

In [ ]:
gruppa3=grupp.query('gruppa==3')

In [ ]:
project=gruppa3.query('event!="project"')

In [ ]:
project = project.groupby(['user_id','event'])['event_datetime'].min()

In [ ]:
project=project.reset_index()

In [ ]:
project['diff'] = project.groupby('user_id')['event_datetime'].diff(1)

In [ ]:
project['diff'] = project['diff']/np.timedelta64(1, 'h')

In [ ]:
project=project.dropna()

In [ ]:
project['diff'] = project['diff'].astype('int')

In [ ]:
project = project.sort_values('user_id')

In [ ]:
project.head()

In [ ]:
aaaa = pd.Series(range(0, len(building)))
plt.figure(figsize=(15,6))
plt.scatter(aaaa, building['diff'])
plt.title('точечный график часов пользователей(Победа над первым врагом) от первой постройки до конца уровня',fontsize=15)
plt.ylabel('Количество часов')
plt.xlabel('Пользователи')
plt.show()

In [ ]:
plt.hist(building['diff'], alpha = 0.9, label='A', bins = 33)
plt.hist(project['diff'], alpha = 0.7, label='B', bins = 33)
plt.legend()
plt.title('Гистограммы кол-ва сыгранных часов по группам Победители и Проектники')
plt.show()

# Вывод: "проектировщики" играют в основном от 200 до 400 часов. "победители" от 100 до 400 часов

In [ ]:
bbbb = pd.Series(range(0, len(project)))
plt.figure(figsize=(15,6))
plt.scatter(bbbb, project['diff'])
plt.title('точечный график часов пользователей(проектники) от первой постройки до конца уровня',fontsize=16)
plt.ylabel('Количество часов')
plt.xlabel('Пользователи')
plt.show()

# Это пользователи и время(колонка y), которое они потратили от первой постройки до конца уровня

In [ ]:
project.shape

In [ ]:
building.shape

In [ ]:
lst = [3951, 1866]
df22 = pd.DataFrame(lst)

# Влияние событий на совершение целевого события

In [ ]:
(df22[0].plot(kind='pie', labels=None, label='', autopct='%1.0f%%', 
        figsize=(10, 10), title='Влияние событий на совершение целевого события', fontsize=20))
labels = ['Победа над первым врагом', 'Реализация проекта']
plt.legend(labels,loc=1)
plt.show()

# Гипотеза различия времени прохождения уровня между пользователями, которые заканчивают уровень через реализацию проекта, и пользователями, которые заканчивают уровень победой над другим игроком.

In [ ]:
A=building['diff']

In [ ]:
A1=building['diff'].mean()

In [ ]:
A2=building['diff'].median()

In [ ]:
A1

In [ ]:
A2

# Где-то 11 дней(победа над первым врагом - "победители")

In [ ]:
B=project['diff']

In [ ]:
B1=project['diff'].mean()

In [ ]:
B2=project['diff'].median()

In [ ]:
B1

In [ ]:
B2

# Где-то 13 дней(реализация проекта - "проектировщики")

In [ ]:
alpha = 0.05
results = st.ttest_ind(A,B)
print('pvalue: {:.2F}'.format(results.pvalue))
if results.pvalue < alpha:
    print('различия времени прохождения уровня между пользователями, которые заканчивают уровень через реализацию проекта, и пользователями, которые заканчивают уровень победой над другим игроком есть!!!')
else:
    print('различия времени прохождения уровня между пользователями, которые заканчивают уровень через реализацию проекта, и пользователями, которые заканчивают уровень победой над другим игроком нет!!!')

# Дополнительная гипотеза: Люди пришедшие с ютуба и яндекса тратят одинаковое количетсво времени на прохождение первого уровня(окончанием проекта)

In [ ]:
gipoteza=pd.merge(project, df2, on='user_id', how='inner')
yandex=gipoteza.query('source=="yandex_direct"')
youtube=gipoteza.query('source=="youtube_channel_reklama"')

In [ ]:
A44=yandex['diff']
b44=youtube['diff']

In [ ]:
alpha = 0.05
results = st.ttest_ind(A44,b44)
print('pvalue: {:.2F}'.format(results.pvalue))
if results.pvalue < alpha:
    print('различия времени прохождения уровня(окончание проекта) пользователями пришедшие с яндекса, и пользователями пришедшие с ютуба, есть!!!')
else:
    print('различия времени прохождения уровня(окончание проекта) пользователями пришедшие с яндекса, и пользователями пришедшие с ютуба, нет!!!')

## Похоже от источников трафика мало что зависит. А значитможно подумать о перераспределении маркетингового бюджета

# Общий вывод: Нужно дать большую мотивацию для созидания(типа никогда по сути не достроишь, потому всегда есть что исправлять, добавлять, улучшать). формула такова - много вариантов созидания для разнообразной(разные подходы) победы над первым врагом. либо много вариантов созидания для разнообразного завершения проекта. оплату рекомендуется вводить ТОЛЬКО после завершения первого уровня.

# и да, максимально вкладываемся в ютуб!

## Дополнение: Включать монетизацию нужно, когда игрок вошёл в игру так сказать. До тех пор НИЧЕГО не должно говорить про то,что нужно что то заплатить.

## Дополнение#2 : У нас есть 2 главных параметра. Это откуда пришли и сколько "вошло в игру". Поэтому я и написал - вкладываемся в ютуб. А строить модель монетизации на тех, кто по сути не вошёл в игру глупо. Нужно строить её на процессе игры, тех кто вошёл в игру. 

## Материалы
- презентация - https://disk.yandex.ru/i/rwVgzQsQUJ8pwg
- дэшборд - https://public.tableau.com/profile/anar3519#!/vizhome/Book2_16159950929170/Dashboard1?publish=yes